Import dependencies

In [130]:
# import gym
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

#import helpers
import numpy as np
import random
import os

#import stable baselines
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

Types of Spaces

In [131]:
Discrete(3).sample()

1

In [132]:
Box(0, 1, shape=(3,)).sample()

array([0.33654493, 0.7136001 , 0.6445912 ], dtype=float32)

In [133]:
Tuple((Discrete(3), Box(0,1, shape=(3,)))).sample()

(2, array([0.72962904, 0.27369156, 0.6415968 ], dtype=float32))

In [134]:
Dict({'height':Discrete(2), "speed":Box(0, 100, shape=(1,))}).sample()

OrderedDict([('height', 1), ('speed', array([17.37748], dtype=float32))])

In [135]:
MultiBinary(4).sample()

array([1, 0, 1, 0], dtype=int8)

In [136]:
MultiDiscrete([5,10,200]).sample()

array([ 1,  1, 24])

Building environment

In [137]:
# keep water temperature between 37 and 39 *C
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low=np.array([0]), high=np.array([100])) # Box(low=0, high=100, shape=(1,))
        self.state = 38 + random.randint(-3, 3) # initial state
        self.shower_length = 60 # frames 

    def step(self, action):
        # apply temp adj
        self.state += action - 1    # increaser, do nothing or decrease temperature

        # decrease shower time
        self.shower_length -= 1

        # calculate reward
        if self.state >= 37 and self.state <= 39:
            reward = 1
        else:
            reward = -1
        
        if self.shower_length <= 0:
            done = True
        else:
            done = False

        info = {}

        return self.state, reward, done, info
    
    def render(self):
        pass

    def reset(self):
        self.state = 38 + random.randint(-3, 3)
        self.shower_length = 60
        return self.state

In [138]:
env = ShowerEnv()

/home/kuba/.local/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [139]:
env.observation_space.sample()

array([32.432865], dtype=float32)

In [140]:
env.action_space.sample()

2

Test environment

In [141]:
episodes = 5
for episode in range(1, episodes + 1):
    observation = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()     # Using model here
        observation, reward, done, info = env.step(action)
        score += reward
    print("Episode {} Score {}".format(episode, score))
env.close()

Episode 1 Score -54
Episode 2 Score -60
Episode 3 Score -20
Episode 4 Score 2
Episode 5 Score -46


Train model

In [142]:
log_path = os.path.join('Training', 'Logs')
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [143]:
model.learn(total_timesteps=50000)

Logging to Training/Logs/PPO_4
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -31.5    |
| time/              |          |
|    fps             | 2012     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -34.1        |
| time/                   |              |
|    fps                  | 1265         |
|    iterations           | 2            |
|    time_elapsed         | 3            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0043157097 |
|    clip_fraction        | 0.0476       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.09        |
|    explained_variance   |

In [144]:
shower_path = os.path.join('Training', 'Saved_models', 'Shower_model_ppo')
model.save(shower_path)

In [145]:
del model

In [146]:
model = PPO.load(shower_path)

In [159]:
evaluate_policy(model, env, n_eval_episodes=10)

(59.2, 0.9797958971132712)